In [42]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import regex as re

Fil = pd.read_pickle('../data/raw/Fil.pkl')
Dokument = pd.read_pickle('../data/raw/Dokument.pkl')

In [8]:
Fil.iloc[0]

id                                                               158
dokumentid                                                     11432
titel              Evaluering af lov om friplejeboliger_endelig v...
versionsdato                                     2014-09-04 15:44:54
filurl             https://www.ft.dk/samling/20131/almdel/BYB/bil...
opdateringsdato                           2020-02-25 10:46:59.490000
variantkode                                                        P
format                                                           PDF
Name: 0, dtype: object

In [14]:
(len(Dokument)-335834)/len(Dokument)

0.6144372766275976

In [11]:
Dokument['kategoriid'].value_counts()[0:20]
#man kan nok godt fjerne udvalgsspørsmål (36), samrådsspørgsmål (37),  foreløbigt svar (30) og  §20 spørgsmål (14)
#det sparer 245123 +23212+ 53136 + 15763 = 335834 = ca. 40% af dokumenterne

C:\Users\rune7\AppData\Local\Temp\ipykernel_30548\2156208293.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  Dokumennt['kategoriid'].value_counts()[0:20]


22    280349
36    245123
15     81565
14     53137
53     43353
30     23212
56     21811
55     20479
37     15763
51     11413
31      8886
23      4311
46      4306
58      4011
35      3572
24      3451
18      3245
25      3190
65      2787
11      2750
Name: kategoriid, dtype: int64

In [15]:
unwanted_categories = [36,37,30,14]

In [18]:
Filtered_dok = Dokument[~Dokument['kategoriid'].isin(unwanted_categories)]
len(Filtered_dok)

533788

In [36]:
Filtered_fil = Fil[Fil['dokumentid'].isin(Filtered_dok['id'])]
len(Filtered_fil)

326384

In [39]:
#pickle the filtered dataframes
Filtered_dok.to_pickle('../data/raw/Filtered_dok.pkl')
Filtered_fil.to_pickle('../data/raw/Filtered_fil.pkl')

In [21]:
len(Fil)

491215

In [35]:
Filtered_dok[Filtered_dok['id'] == 11550]

,id,typeid,kategoriid,statusid,offentlighedskode,titel,dato,modtagelsesdato,frigivelsesdato,paragraf,paragrafnummer,spørgsmålsordlyd,spørgsmålstitel,spørgsmålsid,procedurenummer,grundnotatstatus,dagsordenudgavenummer,opdateringsdato


In [33]:
Dokumennt[Dokumennt['id']==11550]

,id,typeid,kategoriid,statusid,offentlighedskode,titel,dato,modtagelsesdato,frigivelsesdato,paragraf,paragrafnummer,spørgsmålsordlyd,spørgsmålstitel,spørgsmålsid,procedurenummer,grundnotatstatus,dagsordenudgavenummer,opdateringsdato
11362,11550,13,36,1,O,Spm. om ministeren vil redegøre for praksis og...,2014-07-03,None,2014-07-03 14:53:05,,,None,None,<NA>,,,None,2014-09-17 04:22:30.180


In [38]:
Filtered_fil['filurl'][0]

'https://www.ft.dk/samling/20131/almdel/BYB/bilag/89/1396405.pdf'

In [7]:
Dokumennt['statusid'].value_counts()

1    718749
3    142933
2      7445
4      1896
Name: statusid, dtype: int64

In [3]:
with open('../data/processed/158.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
text

'\n\nBy- og Boligudvalget 2013-14BYB Alm.del Bilag 89Offentligt\n\nTil\nMinisteriet for By, Bolig og Landdistrikter\nDato\nJanuar 2012\nVersion\n1.0\nEVALUERING AF LOV OM\nFRIPLEJEBOLIGER\n\nEVALUERING AF LOV OM FRIPLEJEBOLIGER\nRambøll\nHannemanns Allé 53\nDK-2300 København S\nT +45 5161 1000\nF +45 5161 1001\nwww.ramboll-management.dk\n\nEVALUERING AF LOV OM FRIPLEJEBOLIGER\nINDHOLD\n1.\n1.1\n1.2\n1.3\n2.\n2.1\n2.2\n2.3\n2.4\n3.\n3.1\n3.2\n3.2.1\n3.2.2\n3.2.3\n3.2.4\n3.2.5\n4.\n4.1\n4.2\n4.2.1\n4.2.2\n4.2.3\n4.3\n4.3.1\n4.3.2\n4.3.3\n4.4\n4.5\n4.6\n5.\n5.1\n5.2\n5.3\n5.4\n5.5\n5.6\n5.7\n5.8\n5.9\n6.\n6.1\n6.2\n6.2.1\n6.2.2\n6.2.3\n6.3\nIndledning\nFormål\nEvalueringsdesign\nRapportens struktur og læsevejledning\nResumé\nLovgivningens indretning\nBorgernes anvendelse af det frie valg\nKonkurrencesituationen\nKommunernes administration\nLovgivningens formål og indhold\nLovens formål\nLovens indhold\nCertificering\nEtablering\nDrift og afregning af service og pleje\nTilsyn\nUdbud og afh

In [57]:
len(text)
beginning = text[0:1000]

In [58]:
from danlp.models import load_bert_base_model
model = load_bert_base_model()
vecs_embedding, sentence_embedding, tokenized_text =model.embed_text(beginning)

Some weights of the model checkpoint at C:\Users\rune7\.danlp\bert.botxo.pytorch were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [65]:
tokenized_text

['folketingets',
 'lov',
 '##sekret',
 '##ariat',
 'erhvervs',
 '-',
 'og',
 'vækst',
 '##ministeren',
 '15',
 '.',
 'januar',
 '2014',
 'vedlagt',
 'fremsender',
 'jeg',
 'min',
 'besvarelse',
 'af',
 'spørgsm',
 '##al',
 's',
 '##73',
 '##9',
 'stillet',
 'af',
 'jakob',
 'engel',
 '-',
 'schmidt',
 '(',
 'v',
 ')',
 'den',
 '7',
 '.',
 'januar',
 '2014',
 '.',
 'med',
 'venlig',
 'hilsen',
 'henrik',
 'sas',
 '##s',
 'larsen',
 'erhvervs',
 '-',
 'og',
 'vækst',
 '##ministeriet',
 'slots',
 '##holms',
 '##gade',
 '10',
 '-',
 '12',
 '12',
 '##16',
 'københavn',
 'k',
 'tlf',
 '.',
 '33',
 '92',
 '33',
 '50',
 'fax',
 '33',
 '12',
 '37',
 '78',
 'cvr',
 '-',
 'nr',
 '10',
 '09',
 '24',
 '85',
 'ev',
 '##m',
 '[UNK]',
 'ev',
 '##m',
 '.',
 'dk',
 'www',
 '.',
 'ev',
 '##m',
 '.',
 'dk',
 '2013',
 '-',
 '14',
 's',
 '73',
 '##9',
 'endeligt',
 'svar',
 'offentligt',
 'erhvervs',
 '-',
 'og',
 'vækst',
 '##ministeren',
 '15',
 '.',
 'januar',
 '2014',
 'besvarelse',
 'af',
 'spørgsm',
 

In [42]:
print(text)

Folketingets Lovsekretariat
ERHVERVS- OG 
VÆKSTMINISTEREN
15. januar 2014
Vedlagt fremsender jeg min besvarelse af spørgsmål S739 stillet af Jakob 
Engel-Schmidt (V)den 7. januar 2014.
Med venlig hilsen
Henrik Sass Larsen
ERHVERVS- OG 
VÆKSTMINISTERIET
Slotsholmsgade 10-12
1216 København K
Tlf.
33 92 33 50
Fax
33 12 37 78
CVR-nr
10 09 24 85
evm@evm.dk
www.evm.dk
2013-14
S 739 endeligt svar
Offentligt
ERHVERVS- OG 
VÆKSTMINISTEREN
15. januar 2014
Besvarelse af spørgsmål S739stillet af Jakob Engel-Schmidt (V)den 7. 
januar 2014.
Spørgsmål:
Hvad vil ministeren gøre for at stoppe den negative udvikling i antallet af 
nyetablerede virksomheder, der jf. artiklen ”Nystartede virksomheder 
rammer ti års bund” i Børsen den 6. januar 2014 satte bundrekord i 2013?
Svar:
Under højkonjunkturen i årene op til finanskrisens start steg antallet af ny-
startede virksomheder kraftigt. Den efterfølgende langvarige lavkonjunktur 
har i sagens natur gjort det vanskeligere at starte ny virksomhed.
Regeringe

In [71]:
#open bad files
import pickle
with open('../data/raw/bad_files.pkl', 'rb') as f:
    bad_files = pickle.load(f)

len(bad_files)

19665

In [72]:
bad_files

defaultdict(None,
            {466: 403,
             2456: 403,
             1180: 403,
             1746: 403,
             2957: 403,
             3009: 403,
             2923: 403,
             3168: 403,
             3175: 403,
             7418: requests.exceptions.MissingSchema("Invalid URL '/index.htm': No scheme supplied. Perhaps you meant http:///index.htm?"),
             8012: requests.exceptions.MissingSchema("Invalid URL '/index.htm': No scheme supplied. Perhaps you meant http:///index.htm?"),
             3197: 403,
             3196: 403,
             2873: 403,
             20028: 403,
             20030: 403,
             20029: 403,
             35605: 403,
             462: 500,
             2875: 500,
             37955: 403,
             2853: 500,
             37703: 403,
             4607: 404,
             4653: 404,
             9404: 500,
             4694: 404,
             4664: 404,
             4642: 404,
             9413: 500,
             8792: 404,
  

In [81]:
import requests
import regex as re
from tqdm import tqdm
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict
import PyPDF2
from io import BytesIO
from bs4 import BeautifulSoup


In [85]:
def extract_text_from_html(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    return soup.get_text()

def extract_text_from_pdf(pdf_doc):
    # Create a PdfFileReader object
    pdf_reader = PyPDF2.PdfFileReader(BytesIO(pdf_doc))

    # Initialize an empty string to store the extracted text
    extracted_text = ""

    # Iterate through all the pages in the PDF and extract the text
    for page_num in range(pdf_reader.numPages):
        page = pdf_reader.getPage(page_num)
        extracted_text += page.extractText()

    return extracted_text

def process_text(text):
    #removing the sentence "PDF to HTML - Convert PDF files to HTML files"
    text = re.sub(r'PDF to HTML - Convert PDF files to HTML files', '', text)
    #removing newlines when there are more than 2 in a row
    text = re.sub(r'\n{3,}', '\n\n', text)
    return text

output_folder = '../data/processed/'


In [86]:
for key, val in bad_files.items():
    if val == 403:
        try:
            url = Fil.loc[key]['filurl']
            file_path = os.path.join(output_folder, f"{str(key)}.txt")
            response = requests.get(url)
            if response.status_code == 200:
                html_doc = response.text
                raw_text = extract_text_from_html(html_doc)
                text = process_text(raw_text)
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(text)
        except:
            pass
    else:
        pass

In [43]:
Dokument[Dokument['kategoriid'] == 14]

,id,typeid,kategoriid,statusid,offentlighedskode,titel,dato,modtagelsesdato,frigivelsesdato,paragraf,paragrafnummer,spørgsmålsordlyd,spørgsmålstitel,spørgsmålsid,procedurenummer,grundnotatstatus,dagsordenudgavenummer,opdateringsdato
1,2,16,14,1,O,"Er ministeren enig i vurderingen af, at grønla...",2014-01-02,None,2014-01-02 11:36:24,,,None,None,<NA>,,,None,2015-05-29 11:11:34.047
3,4,16,14,1,O,"Hvad agter ministeren at foretage sig, så kost...",2014-01-02,None,2014-01-20 10:12:24,,,None,None,<NA>,,,None,2015-05-29 11:11:32.563
15,16,16,14,1,O,"Mener ministeren, at det er fair, at det offen...",2014-01-03,None,2014-01-20 10:14:03,,,None,None,<NA>,,,None,2015-05-29 11:11:15.453
17,18,16,14,1,O,"Mener ministeren, at det vil være en god ide a...",2014-01-03,None,2014-01-20 10:13:46,,,None,None,<NA>,,,None,2015-05-29 11:11:30.750
29,31,16,14,1,O,"Vil udenrigsministeren i lyset af, at danske m...",2014-01-03,None,2014-01-03 13:53:04,,,None,None,<NA>,,,None,2015-05-29 11:11:26.343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870983,1024377,16,14,1,O,"Mener ministeren, at det er i overensstemmelse...",2023-02-27,2023-02-27 00:00:00,2023-02-27 17:12:17,,,None,None,<NA>,,,None,2023-02-27 17:12:29.357
870984,1024378,16,14,1,O,"Har ministeren overvejet, hvordan det vil påvi...",2023-02-27,2023-02-27 00:00:00,2023-02-27 16:27:47,,,None,None,<NA>,,,None,2023-02-27 16:28:00.090
870997,1024402,16,14,1,O,"Er ministeren enig i, at en kilometerafgift fo...",2023-02-27,2023-02-27 00:00:00,2023-02-27 17:16:27,,,None,None,<NA>,,,None,2023-02-27 17:16:39.167
871001,1024407,16,14,1,O,I lyset af at ministerens parti ønsker et Danm...,2023-02-27,2023-02-27 00:00:00,2023-02-27 17:18:11,,,None,None,<NA>,,,None,2023-02-27 17:18:23.077


In [47]:
#Taking the 'id' of all the documents with categoryid 14

p20_q_ids = Dokument[Dokument['kategoriid'] == 14]['id'].tolist()

In [48]:
#TAking all the Documet ids that have a "spørgsmålsid" that is in the list of p20_q_ids
p20_answer_ids = Dokument[Dokument['spørgsmålsid'].isin(p20_q_ids)]['id'].tolist()
p20_answer_ids

[3,
 5,
 17,
 19,
 32,
 34,
 36,
 38,
 40,
 42,
 44,
 46,
 62,
 65,
 120,
 122,
 124,
 126,
 128,
 134,
 141,
 143,
 149,
 154,
 156,
 158,
 160,
 173,
 175,
 177,
 179,
 183,
 188,
 190,
 204,
 206,
 208,
 210,
 213,
 217,
 219,
 223,
 225,
 242,
 244,
 246,
 248,
 250,
 253,
 256,
 258,
 260,
 262,
 264,
 266,
 268,
 273,
 275,
 279,
 281,
 283,
 285,
 288,
 290,
 303,
 305,
 333,
 335,
 337,
 350,
 353,
 355,
 360,
 362,
 375,
 377,
 379,
 381,
 383,
 385,
 390,
 392,
 394,
 396,
 398,
 401,
 403,
 405,
 413,
 415,
 417,
 421,
 423,
 425,
 427,
 432,
 435,
 438,
 440,
 442,
 444,
 446,
 448,
 451,
 453,
 474,
 476,
 478,
 481,
 483,
 500,
 504,
 507,
 511,
 515,
 524,
 529,
 534,
 536,
 554,
 556,
 558,
 560,
 611,
 613,
 617,
 619,
 621,
 623,
 625,
 627,
 629,
 631,
 633,
 635,
 637,
 650,
 652,
 654,
 656,
 658,
 660,
 662,
 664,
 666,
 668,
 670,
 672,
 674,
 676,
 678,
 683,
 687,
 689,
 693,
 695,
 697,
 703,
 705,
 707,
 709,
 711,
 716,
 718,
 720,
 723,
 728,
 730,
 732,
 7

In [49]:
Dokument[Dokument['id'] == 3]

,id,typeid,kategoriid,statusid,offentlighedskode,titel,dato,modtagelsesdato,frigivelsesdato,paragraf,paragrafnummer,spørgsmålsordlyd,spørgsmålstitel,spørgsmålsid,procedurenummer,grundnotatstatus,dagsordenudgavenummer,opdateringsdato
2,3,17,22,1,O,Svar på spm. nr. S 725: Er ministeren enig i v...,2014-01-09,2014-01-09 00:00:00,2014-01-10 09:26:05,,,,"Er ministeren enig i vurderingen af, at grønla...",2,,,None,2014-09-30 16:15:46.607


In [51]:
q_ids = Dokument[Dokument['kategoriid'] == 13]['id'].tolist()
q_ids

[40399,
 280201,
 280203,
 280204,
 280205,
 280206,
 280214,
 280215,
 280216,
 280217,
 282525,
 284982,
 285083,
 285090,
 285178,
 285190,
 285782,
 285783,
 285784,
 285785,
 285787,
 297296,
 322667,
 324609,
 324610,
 324653,
 329870,
 330136,
 330137,
 330241,
 330242,
 330251,
 330313,
 330357,
 330358,
 330447,
 330468,
 330469,
 331048,
 331224,
 331225,
 331282,
 331283,
 331331,
 331332,
 331338,
 331358,
 331396,
 331412,
 331421,
 331640,
 331778,
 332214,
 332215,
 332216,
 332908,
 332978,
 333340,
 333978,
 334053,
 334054,
 334055,
 334480,
 334481,
 334482,
 335071,
 335085,
 335106,
 335109,
 336857,
 337050,
 337208,
 339698,
 339718,
 339719,
 341082,
 341496,
 341713,
 344582,
 346063,
 346226,
 347672,
 347673,
 348029,
 348249,
 350086,
 351050,
 351457,
 353582,
 353583,
 354076,
 354078,
 354098,
 356363,
 356685,
 362169,
 393977,
 394362,
 394731,
 394732,
 394733,
 394734,
 394735,
 394872,
 394873,
 395104,
 395105,
 395106,
 395107,
 395287,
 395288,
 3

In [66]:
#taking dokument have a spørgsmålsid that is not none and a kategoriid == 22
Dokument[~Dokument['spørgsmålsid'].isna() & (Dokument['kategoriid'] == 22)][50:100]

,id,typeid,kategoriid,statusid,offentlighedskode,titel,dato,modtagelsesdato,frigivelsesdato,paragraf,paragrafnummer,spørgsmålsordlyd,spørgsmålstitel,spørgsmålsid,procedurenummer,grundnotatstatus,dagsordenudgavenummer,opdateringsdato
217,219,17,22,1,O,"Svar på spm. nr. S 778: Mener ministeren, at d...",2014-01-24,2014-01-24 00:00:00,2014-01-24 10:42:26,,,,"Mener ministeren, at det vil være hensigtsmæss...",218,,,None,2015-05-29 08:56:13.623
221,223,17,22,1,O,Svar på spm. nr. S 779: Vil ministeren bidrage...,2014-01-24,2014-01-24 00:00:00,2014-01-24 10:36:03,,,,"Vil ministeren bidrage til en løsning, så Jazz...",222,,,None,2015-05-29 08:56:31.500
223,225,17,22,1,O,Svar på spm. nr. S 780: Hvad vil ministeren gø...,2014-01-24,2014-01-24 00:00:00,2014-01-24 15:17:25,,,,Hvad vil ministeren gøre for at hjælpe de borg...,224,,,None,2015-05-29 08:58:04.017
240,242,17,22,1,O,Svar på spm. nr. S 795: Agter ministeren i lys...,2014-01-27,2014-01-27 00:00:00,2014-01-27 11:58:49,,,,Agter ministeren i lyset af de store forskelle...,241,,,None,2015-05-29 08:57:35.170
242,244,17,22,1,O,Svar på spm. nr. S 796: Hvorfor kommer regerin...,2014-01-27,2014-01-27 00:00:00,2014-01-27 15:58:07,,,,Hvorfor kommer regeringen med en ændring af be...,243,,,None,2015-05-29 08:55:41.577
244,246,17,22,1,O,Svar på spm. nr. S 797: Betyder lovforslaget o...,2014-01-27,2014-01-27 00:00:00,2014-01-27 15:55:59,,,,Betyder lovforslaget om ændring af beredskabsl...,245,,,None,2015-05-29 08:55:22.623
246,248,17,22,1,O,Svar på spm. nr. S 798: Er det regeringens pol...,2014-01-28,2014-01-28 00:00:00,2014-01-28 13:26:39,,,,"Er det regeringens politik, når Socialdemokrat...",247,,,None,2015-05-29 08:59:04.517
248,250,17,22,1,O,"Svar på spm. nr. S 799: Mener ministeren, at d...",2014-01-28,2014-01-28 00:00:00,2014-01-28 13:46:15,,,,"Mener ministeren, at det vil øge lediges incit...",249,,,None,2015-05-29 08:59:33.967
251,253,17,22,1,O,Svar på spm. nr. S 800: Er udenrigsministeren ...,2014-01-27,2014-01-27 00:00:00,2014-01-27 16:08:27,,,,"Er udenrigsministeren enig i, at den kurdiske ...",252,,,None,2015-05-29 08:58:28.657
254,256,17,22,1,O,Svar på spm. nr. S 801: Er ministeren enig med...,2014-01-27,2014-01-27 00:00:00,2014-01-27 15:46:52,,,,Er ministeren enig med Socialdemokraternes lig...,255,,,None,2015-05-29 08:54:55.123
